In [1]:
import cv2

img = cv2.imread("../input/tire-texture-image-recognition/Tire Textures/testing_data/cracked/Cracked-105.jpg")

img.shape

In [2]:
import matplotlib.pyplot as plt

plt.imshow(img)

In [3]:
train_dir = "../input/tire-texture-image-recognition/Tire Textures/training_data"
test_dir = "../input/tire-texture-image-recognition/Tire Textures/testing_data"

In [4]:
#processing the dataset

import tensorflow as tf

train_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale= 1/255.,                                                        
                                                           rotation_range=0.2,
                                                           width_shift_range=0.2,
                                                           height_shift_range=0.2,
                                                           zoom_range = 0.2, 
                                                           horizontal_flip=True,
                                                            )

test_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale= 1/255.)


In [15]:
train_data = train_gen.flow_from_directory(train_dir,
                                            target_size = (224,224),
                                            class_mode = "categorical",
                                            batch_size = 32
                                            )
test_data = test_gen.flow_from_directory(test_dir,
                                          target_size = (224,224),
                                          class_mode = "categorical",
                                          batch_size = 32
                                          )

In [16]:
label_name = list(train_data.class_indices.keys())

label_name

In [17]:
image,label = train_data.next()

print(image.shape)
print(label.shape)


In [9]:
import matplotlib.pyplot as plt
import numpy as np
plt.figure(figsize=(14,14))

for i in range(9):
  image,label = train_data.next()
 
  plt.subplot(3,3,i+1)
  plt.imshow(image[i])
  plt.title(label_name[tf.argmax(label[i])])
  plt.axis("off")


In [44]:
# building s CNN model

import tensorflow as tf

from tensorflow.keras import layers

model = tf.keras.Sequential([
                             layers.Conv2D(filters= 13, kernel_size= 2, activation="relu", input_shape=(224,224,3)),                             
                             layers.Conv2D(filters= 13, kernel_size= 2, activation="relu"),
                             layers.Dropout(0.2),
                             layers.MaxPooling2D(pool_size= 2),
                             
                             layers.Conv2D(filters= 13, kernel_size = 2, activation= "relu"),                             
                             layers.Conv2D(filters= 13, kernel_size= 2, activation= "relu"),  
                             layers.Dropout(0.2),
                             layers.MaxPooling2D(pool_size= 2),

                             layers.Conv2D(filters= 13, kernel_size = 2, activation= "relu"),                             
                             layers.Conv2D(filters= 13, kernel_size= 2, activation= "relu"),
                             layers.Dropout(0.2),
                             layers.MaxPooling2D(pool_size= 2),
                             layers.Flatten(),

                             layers.Dense(1024, activation= "relu"),
                             layers.Dropout(0.2),
                             layers.Dense(1024, activation = "relu"),
                             layers.Dropout(0.2),
                             layers.Dense(1024, activation= "relu"),


                             layers.Dense(2, activation="softmax")
                            
])


In [45]:
model.summary()

In [46]:
# compiling the model

model.compile(
    loss = tf.keras.losses.categorical_crossentropy,
    optimizer = tf.keras.optimizers.Adam(learning_rate= 0.001),
    metrics = ["accuracy"]
)

In [47]:
# fitting the data to the model

model.fit(train_data,
          epochs = 10,
          steps_per_epoch = len(train_data),
          validation_data = test_data,
          validation_steps = len(test_data)
          )

In [48]:
# resnet model using transfer learning

import tensorflow as tf

resnet_base = tf.keras.applications.ResNet50V2(include_top= False)

resnet_base.trainable = False

In [49]:
# building the resnet model
import tensorflow as tf
from tensorflow.keras import layers

inputs = tf.keras.Input(shape=(224,224,3))

x = resnet_base(inputs)

x = layers.GlobalAveragePooling2D()(x)

outputs = layers.Dense(2, activation = "softmax")(x)

resnet_model = tf.keras.Model(inputs,outputs)


In [50]:
# compiling the model

resnet_model.compile(
    loss = tf.keras.losses.categorical_crossentropy,
    optimizer = tf.keras.optimizers.Adam(learning_rate= 0.001),
    metrics = ["accuracy"]
)

In [51]:
# fitting data to the model

resnet_model.fit(
    train_data,
    epochs = 5,
    steps_per_epoch = len(train_data),
    validation_data = test_data,
    validation_steps = len(test_data)
)

In [62]:
# Tuning the model

resnet_base.trainble = True

for layer in resnet_base.layers[:-3]:
  resnet_base.trainable = False


resnet_model.compile(
     loss = tf.keras.losses.categorical_crossentropy,
    optimizer = tf.keras.optimizers.Adam(learning_rate= 0.001),
    metrics = ["accuracy"]
)

resnet_model.fit(
    train_data,
    epochs = 10,
    steps_per_epoch = len(train_data),
    validation_data = test_data,
    validation_steps = len(test_data)
)

In [64]:
resnet_model_evaluation = resnet_model.evaluate(test_data)

In [66]:
print(f"Resnet Model Accuracy: {resnet_model_evaluation[1] * 100: 0.2f} %")

In [53]:
# processing data for efficientnet model
import tensorflow as tf

eff_train = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                image_size = (224,224),
                                                                label_mode = "categorical",
                                                                batch_size = 32                                                            
                                                                )

eff_test = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                               image_size = (224,224),
                                                               label_mode = "categorical",
                                                               batch_size = 32
                                                               )


In [54]:
for image,label in eff_test.take(1):
  print(image.shape)
  print(label.shape)
  break

In [55]:
# building a model on efficientnet
import tensorflow as tf

effnet_base = tf.keras.applications.EfficientNetB5(include_top= False)

effnet_base.trainable = False

In [56]:
# building the resnet model
import tensorflow as tf
from tensorflow.keras import layers

inputs = tf.keras.Input(shape=(224,224,3))

x = effnet_base(inputs)

x = layers.GlobalAveragePooling2D()(x)

outputs = layers.Dense(2, activation = "softmax")(x)

effnet_model = tf.keras.Model(inputs,outputs)


In [57]:
# compiling the model

effnet_model.compile(
    loss = tf.keras.losses.categorical_crossentropy,
    optimizer = tf.keras.optimizers.Adam(learning_rate= 0.001),
    metrics = ["accuracy"]
)

In [58]:
# fitting data to the model

effnet_model.fit(
    eff_train,
    epochs = 5,
    steps_per_epoch = len(eff_train),
    validation_data = eff_test,
    validation_steps = len(eff_test)
)

In [70]:
# Tuning the model

effnet_base.trainble = True

for layer in effnet_base.layers[:-3]:
  effnet_base.trainable = False


effnet_model.compile(
     loss = tf.keras.losses.categorical_crossentropy,
    optimizer = tf.keras.optimizers.Adam(learning_rate= 0.001),
    metrics = ["accuracy"]
)

effnet_model.fit(
    eff_train,
    epochs = 10,
    steps_per_epoch = len(eff_train),
    validation_data = eff_test,
    validation_steps = len(eff_test)
)

In [72]:
effnet_evaluation = effnet_model.evaluate(eff_test)

In [73]:
print(f"EfficientNet Model Accuarcy: {effnet_evaluation[1] * 100 :0.2f} %")

In [74]:
test_label = list(test_data.class_indices.keys())

test_label

In [75]:
# Resnet model prediction and visualizing

import matplotlib.pyplot as plt
import tensorflow as tf

plt.figure(figsize=(14,14))

for i in range(9):
    
    image,label = test_data.next()
    resnet_pred = resnet_model.predict(image)
    plt.subplot(3,3,i+1)
    plt.imshow(image[i])
    plt.title(f"Prediction : {test_label[tf.argmax(resnet_pred[i])]}\n Original : {test_label[tf.argmax(label[i])]}")
    plt.axis("off")



In [77]:
eff_label = eff_test.class_names
eff_label

In [79]:
# Efficientnet model prediction and visualizing

import matplotlib.pyplot as plt
import tensorflow as tf

plt.figure(figsize=(14,14))

for image, label in eff_test.take(1):
    
    eff_pred = effnet_model.predict(image)
    for i in range(9):   
               
        plt.subplot(3,3,i+1)
        plt.imshow(image[i].numpy().astype("uint8"))
        plt.title(f"Prediction : {eff_label[tf.argmax(eff_pred[i])]}\n Original : {eff_label[tf.argmax(label[i])]}")
        plt.axis("off")